In [ ]:
import numpy as np
import pandas as pd
import pickle


PATH = "CMADS_1979_2018_(14610_130_9).npy"
X = np.load(PATH)  
assert X.shape == (14610, 130, 9), f"Unexpected shape: {X.shape}"

dates = pd.date_range("1979-01-01", "2018-12-31", freq="D")


P_hpa = X[:, :, 0]
Tmean = X[:, :, 1]
Tmax = X[:, :, 2]
Tmin = X[:, :, 3]
Prec = X[:, :, 4]
Rs = X[:, :, 6]


Prec = np.nan_to_num(Prec, nan=0.0)
Rs = np.nan_to_num(Rs, nan=0.0)
Tmean = np.nan_to_num(Tmean, nan=0.0)
Tmax = np.nan_to_num(Tmax, nan=0.0)
Tmin = np.nan_to_num(Tmin, nan=0.0)
P_hpa = np.nan_to_num(P_hpa, nan=1013.25)


Prec = np.clip(Prec, 0.0, None)      
Rs = np.clip(Rs, 0.0, None)          


def et0_makkink_mm_day(T_c, Rs_MJ_m2_day, P_hpa):

    P_kpa = P_hpa * 0.1  

    
    es = 0.6108 * np.exp(17.27 * T_c / (T_c + 237.3))
    
    delta = 4098.0 * es / ((T_c + 237.3) ** 2)
    
    gamma = 0.000665 * P_kpa
    
    lam = 2.45

    
    ET0 = 0.61 * (delta / (delta + gamma)) * (Rs_MJ_m2_day / lam) - 0.12
    ET0 = np.clip(ET0, 0.0, None)
    return ET0


ET0 = et0_makkink_mm_day(Tmean, Rs, P_hpa)  

df_prec = pd.DataFrame(Prec, index=dates)     
df_tmean = pd.DataFrame(Tmean, index=dates)   
df_tmin = pd.DataFrame(Tmin, index=dates)     
df_tmax = pd.DataFrame(Tmax, index=dates)     
df_et0  = pd.DataFrame(ET0,  index=dates)     


T_ann = df_tmean.resample("YE").mean().mean(axis=0).to_numpy()


Tb = 10.0
GDD10_daily = np.maximum(Tmean - Tb, 0.0)
df_gdd10 = pd.DataFrame(GDD10_daily, index=dates)
GDD10 = df_gdd10.resample("YE").sum().mean(axis=0).to_numpy()


HeatDays35 = (df_tmax >= 35.0).astype(float).resample("YE").sum().mean(axis=0).to_numpy()



years = sorted(set(dates.year))
n_stations = X.shape[1]
FFP_yearly = np.zeros((len(years), n_stations), dtype=float)

for yi, y in enumerate(years):
    idx = (dates.year == y)
    dts = dates[idx]
    tmin_year = Tmin[idx, :]  

    spring_mask = (dts.month <= 6)  
    autumn_mask = (dts.month >= 7)  
    autumn_start = np.where(autumn_mask)[0][0]  

    for s in range(n_stations):
        sp = np.where((tmin_year[spring_mask, s] < 0.0))[0]
        au = np.where((tmin_year[autumn_mask, s] < 0.0))[0]

        last_spring = sp.max() if len(sp) > 0 else -1
        first_autumn = (autumn_start + au.min()) if len(au) > 0 else tmin_year.shape[0]

        ffp = max(first_autumn - last_spring - 1, 0)
        FFP_yearly[yi, s] = ffp

FFP = FFP_yearly.mean(axis=0)  


P_ann = df_prec.resample("YE").sum().mean(axis=0).to_numpy()  
ET0_ann = df_et0.resample("YE").sum().mean(axis=0).to_numpy() 
AI = P_ann / np.maximum(ET0_ann, 1e-6)  



monthly_prec = df_prec.resample("ME").sum()  
prec_clim_12 = monthly_prec.groupby(monthly_prec.index.month).mean()  
P_ann_clim = prec_clim_12.sum(axis=0).to_numpy()  
P_m = prec_clim_12.to_numpy()  
SI = np.sum(np.abs(P_m - (P_ann_clim / 12.0)[None, :]), axis=0) / np.maximum(P_ann_clim, 1e-6)


def bin_class(x, thresholds):

    return np.digitize(x, thresholds, right=False)



THERMAL_BINS = [1000, 2000, 3000]  
thermal_c = bin_class(GDD10, THERMAL_BINS)


MOIST_BINS = [0.2, 0.5, 0.65, 1.0]  
moist_c = bin_class(AI, MOIST_BINS)


SEASON_BINS = [0.2, 0.4, 0.6, 0.8]  
season_c = bin_class(SI, SEASON_BINS)


zone_id = thermal_c * 100 + moist_c * 10 + season_c  
unique_zones, counts = np.unique(zone_id, return_counts=True)
print(f"[Info] Zones: {len(unique_zones)}")
print("[Info] Top zones (zone_id: count):", sorted(zip(unique_zones.tolist(), counts.tolist()), key=lambda x: -x[1])[:10])

feat = np.vstack([T_ann, GDD10, FFP, HeatDays35, AI, SI]).T  


mu = feat.mean(axis=0)
sd = feat.std(axis=0) + 1e-8
z = (feat - mu) / sd


sq = np.sum(z**2, axis=1, keepdims=True)
dist2 = sq + sq.T - 2.0 * (z @ z.T)
dist2 = np.maximum(dist2, 0.0)

SIGMA = 1.0         
LAMBDA_INTER = 0.15 

sim = np.exp(-dist2 / (2.0 * SIGMA**2))
same_zone = (zone_id[:, None] == zone_id[None, :])

A_prior = sim * (same_zone.astype(float) + (~same_zone).astype(float) * LAMBDA_INTER)
np.fill_diagonal(A_prior, 0.0)  

print("[Info] A_prior:", A_prior.shape, "min/max:", A_prior.min(), A_prior.max())


A_with_I = A_prior + np.eye(n_stations, dtype=float)


deg = A_with_I.sum(axis=1)
D_inv_sqrt = np.diag(1.0 / np.sqrt(np.maximum(deg, 1e-12)))
A_norm = D_inv_sqrt @ A_with_I @ D_inv_sqrt


with open("homozone/A_prior_homozone.pkl", "wb") as f:
    pickle.dump(A_prior.astype(np.float32), f)
np.save("homozone/A_prior_homozone_130x130.npy", A_prior.astype(np.float32))
np.save("homozone/A_norm_homozone_130x130.npy", A_norm.astype(np.float32))
np.save("homozone/zone_id_130.npy", zone_id.astype(np.int32))
np.save("homozone/agro_climate_features_130x6.npy", feat.astype(np.float32))

print("[Saved] homozone/A_prior_homozone_130x130.npy")
print("[Saved] homozone/A_norm_homozone_130x130.npy")
print("[Saved] homozone/zone_id_130.npy")
print("[Saved] homozone/agro_climate_features_130x6.npy")
